# PROJET DEEP LEARNING

## Alpha Digits 

In [1]:
import copy
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
import scipy as sp

# Spécifiez le chemin du fichier .mat
digits = "binaryalphadigs.mat"

# Chargez les données MATLAB
mat_data_digits = scipy.io.loadmat(digits)

# Récupérez les clés du dictionnaire
keys = mat_data_digits.keys()

dat_array = mat_data_digits['dat']

# Affichez les clés
print("Clés du dictionnaire :", keys)

Clés du dictionnaire : dict_keys(['__header__', '__version__', '__globals__', 'dat', 'numclass', 'classlabels', 'classcounts'])


In [2]:
def lire_alpha_digit(filename: str, indices=None):
    mat = sp.io.loadmat(filename, simplify_cells=True)
    bad = mat_data_digits["dat"][indices, :]
    images = np.zeros((bad.size, bad[0, 0].size))
    im = 0  # image index
    for i in range(bad.shape[0]):
        for j in range(bad.shape[1]):
            images[im, :] = bad[i, j].flatten()
            im += 1

    return images

# RBM

In [3]:
import matplotlib.pyplot as plt
import numpy as np


def sigmoid(z):
    return 1 / (1 + np.exp(-z))


def logit(z):
    return np.log(z / (1 - z))


class RBM:
    def __init__(self, p, q):
        self.a = np.zeros(p)
        self.b = np.zeros(q)
        self.W = np.random.normal(size=(p, q)) * np.sqrt(0.01)

    def entree_sortie_rbm(self, v):
        return sigmoid(self.b + np.dot(v, self.W))

    def sortie_entree_rbm(self, h):
        return sigmoid(self.a + np.dot(h, self.W.T))

    def train_rbm(
        self, X, epochs, learning_rate, batch_size, verbose=False
    ):
        #keep_track = 0
        weights = []
        losses = []
        for epoch in range(epochs):
            data_copy = X.copy()
            np.random.shuffle(data_copy)

            for batch in range(0, X.shape[0], batch_size):
                data_batch = data_copy[
                    batch: min(batch + batch_size, X.shape[0]), :
                ]

                v0 = data_batch
                p_h_0 = self.entree_sortie_rbm(v0)
                h_0 = np.random.binomial(1, p_h_0)
                p_v_1 = self.sortie_entree_rbm(h_0)
                v1 = np.random.binomial(1, p_v_1)
                p_h_1 = self.entree_sortie_rbm(v1)

                grad_w = np.dot(v0.T, p_h_0) - np.dot(v1.T, p_h_1)
                grad_a = np.sum(v0 - v1, axis=0)
                grad_b = np.sum(p_h_0 - p_h_1, axis=0)

                self.W += learning_rate / batch_size * grad_w
                self.a += learning_rate / batch_size * grad_a
                self.b += learning_rate / batch_size * grad_b
                
                weights.append(np.mean(self.W))

            h_epoch = self.entree_sortie_rbm(X)
            data_rec = self.sortie_entree_rbm(h_epoch)
            mse = np.sum((data_rec - X) ** 2) / X.size
            losses.append(mse)
            
            # Décommenter les lignes suivantes pour Afficher la fonction de perte
            
            #if keep_track < early_stopping and round(mse, 3) == round(previous_mse, 3):
             #   keep_track += 1
            #elif keep_track == early_stopping:
                #return self
            #if verbose:
             #   print(f"Epoch {epoch+1}/{epochs} - Error: {mse:.3f}")

        #plt.plot(losses)
        #plt.xlabel('epochs')
        #plt.ylabel('loss')
        #plt.title('Evolution of the loss through ' + str(epochs) + ' epochs')
        #plt.show()
        #print("Final loss :", losses[-1])

        #plt.xlabel('epochs')
        #plt.ylabel('mean elements of weight W')
        #plt.plot(weights)
        #plt.show()

        return self

    def generate_image_rbm(self, x_im, y_im, nb_images, nb_gibbs, plot=False):
        p = self.W.shape[0]
        images = np.zeros((nb_images, p))
        for data in range(nb_images):
            v = np.random.binomial(1, 0.5 * np.ones(p))
            for iter_gibbs in range(nb_gibbs):
                h = np.random.binomial(1, self.entree_sortie_rbm(v))
                v = np.random.binomial(1, self.sortie_entree_rbm(h))

            images[data, :] = v

        if plot:
            # Reshape and Plot the generated images
            fig, axes = plt.subplots(1, nb_images, figsize=(10, 2))
            images = images.reshape((nb_images, x_im, y_im))
            for i in range(nb_images):
                axes[i].imshow(images[i], cmap="gray")
                axes[i].axis("off")
            plt.show()
        return images
    

    

## Test RBM

In [4]:
X = lire_alpha_digit(digits, [2])
n_X, p_X = X.shape
q = 100  # number of hidden values
epochs_rbm = 1000
learning_rate = 0.01
batch_size = 10
nb_gibbs = 200

rbm = RBM(p_X, q)
rbm.train_rbm(X, epochs_rbm, learning_rate, batch_size)
rbm.generate_image_rbm(20, 16, 5, nb_gibbs, True)

KeyboardInterrupt: 

# Variations données train

In [ ]:
#configuration
epochs = 1000
learning_rate = 0.1
batch_size = 38
nb_gibbs = 300

# Variation du nombre de caractères à apprendre
characters_sets = [
    [i for i in range(2, 3)],  # One character
    [i for i in range(2, 6)],
    [i for i in range(2, 10)],
    [i for i in range(2, 20)], 
    [i for i in range(2, 36)]  # 35 characters
]

for characters_to_learn in characters_sets:
    # Charger les données pour les caractères spécifiés
    X = lire_alpha_digit(digits, characters_to_learn)
    
    rbm = RBM(p_X, 300)
    rbm.train_rbm(X, epochs, learning_rate, batch_size)
    rbm.generate_image_rbm(20, 16, 5, nb_gibbs, True)


# Variations nombre de neurones

In [ ]:
X = lire_alpha_digit(digits, [3])
nb_neurons = [10, 50, 100, 300]
for q in nb_neurons:
    rbm = RBM(p_X, 100)
    rbm.train_rbm(X, epochs, learning_rate, batch_size)
    rbm.generate_image_rbm(20, 16, 5, nb_gibbs, True)


# DBN

In [ ]:
class DBN:
    def __init__(self, layer_sizes):
        self.layer_sizes: tuple = layer_sizes
        self.dbn: [RBM] = []
        for i in range(len(layer_sizes) - 1):
            rbm = RBM(layer_sizes[i], layer_sizes[i + 1])
            self.dbn.append(rbm)

    def train_dbn(self, X, epochs, learning_rate, batch_size):
        for i in range(len(self.layer_sizes) - 1):
            #print("Pre-training RBM %d..." % i)
            self.dbn[i] = self.dbn[i].train_rbm(
                X=X,
                epochs=epochs,
                learning_rate=learning_rate,
                batch_size=batch_size,
            )

            # Calcul des sorties de l'i-ème RBM pour les données d'entrée
            X = np.random.binomial(1, self.dbn[i].entree_sortie_rbm(X))
        return self

    def generate_image_dbn(self, x_im, y_im, nb_data=1, nb_gibbs=100, plot=False):
        v = self.dbn[-1].generate_image_rbm(x_im, y_im, nb_data, nb_gibbs)
        for i in range(2, len(self.layer_sizes)):
            v = np.random.binomial(1, self.dbn[-i].sortie_entree_rbm(v))

        images = v.reshape((nb_data, x_im, y_im))
        if plot:
            # Reshape and Plot generated images
            fig, axes = plt.subplots(1, nb_data, figsize=(10, 2))
            for i in range(nb_data):
                axes[i].imshow(images[i], cmap="gray")
                axes[i].axis("off")
            #plt.suptitle(f"The config considered {self.layer_sizes}")
            plt.show()
        #return images

# Variations données train

In [ ]:
# Fixed configuration
epochs = 1000
learning_rate = 0.1
batch_size = 38
nb_gibbs = 300

# Variation du nombre de caractères à apprendre
characters_sets = [
    [i for i in range(2, 3)],# One character
    [i for i in range(2, 6)],
    [i for i in range(2, 10)],
    [i for i in range(2, 20)], 
    [i for i in range(2, 36)]  # 35 characters
    # Ajoutez plus d'ensembles au besoin
]

for characters_to_learn in characters_sets:
    # Charger les données pour les caractères spécifiés
    X = lire_alpha_digit(digits, characters_to_learn)
    config = [p_X, 300]
    dbn = DBN(config)
    dbn.train_dbn(X, epochs, learning_rate, batch_size)
    dbn.generate_image_dbn(20, 16, 4, nb_gibbs, True)


# Variation nombre de couches 

In [ ]:
X = lire_alpha_digit(digits, [3])
nb_layers = [1,5,10]
for layer in nb_layers:
    # Define the config for each iteration
    config = [p_X]
    config.extend([200 for x in range(layer)])

    dbn = DBN(config)
    dbn.train_dbn(X, 100, learning_rate, batch_size)
    dbn.generate_image_dbn(20, 16, 4, nb_gibbs, True)


# Variation nombre de neurones 

In [ ]:
nb_neurons = [10,50,100]
for neurons in nb_neurons:
    # Define the config for each iteration
    config = [p_X]
    config.extend([neurons for x in range(2)])

    dbn = DBN(config)
    dbn.train_dbn(X, 100, learning_rate, batch_size)
    dbn.generate_image_dbn(20, 16, 4, nb_gibbs, True)


# DNN

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

def sigmoid_prime(z):
    return z * (1 - z)


def calcul_softmax(rbm: RBM, data):
    z = np.array(rbm.b) + np.dot(data, rbm.W)
    return np.exp(z) / np.sum(np.exp(z), axis=1, keepdims=True)


class DNN:
    def __init__(self, config, output_dim=10):

        self.config: tuple = config
        self.num_layers: int = len(self.config)  # number of layers except output

        # un DNN est un DBN avec une couche de classification supplémentaire
        # dernier RBM du DBN pour la classification → on ne définit pas "rbm.a".
        self.dbn: DBN = DBN(config)
        self.classification: RBM = RBM(config[-1], output_dim)
        self.pretrained: bool = False  # check if model is pretrained
        self.fitted: bool = False  # check if model is fitted

    def pretrain_dnn(self, data, epochs=100, learning_rate=0.1, batch_size=100):
        self.dbn.train_dbn(data, epochs, learning_rate, batch_size)
        self.pretrained = True
        return self

    def entree_sortie_network(self, data):
        v = data.copy()
        results = [v]  # Couche d'entrée
        for i in range(self.num_layers - 1):
            p_h = self.dbn.dbn[i].entree_sortie_rbm(v)
            v = np.random.binomial(1, p_h)
            results.append(p_h)

        # Compute the probabilities
        softmax_probas = calcul_softmax(self.classification, v)
        results.append(softmax_probas)
        return results

    def backward_propagation(self, data,labels,epochs=100,learning_rate=0.1,batch_size=100,early_stopping=5,verbose=True,plot=True,):
        keep_track = 0
        train_loss = 100
        loss_batches, loss = [], []

        for epoch in range(epochs):
            data_copy = data.copy()
            labels_copy = pd.get_dummies(labels.copy())
            data_copy, labels_copy = shuffle(data_copy, labels_copy)

            for batch in range(0, data.shape[0], batch_size):
                data_batch = data_copy[
                    batch: min(batch + batch_size, data.shape[0]), :
                ]
                labels_batch = labels_copy[
                    batch: min(batch + batch_size, data.shape[0])
                ]
                # Forward pass
                activations = self.entree_sortie_network(data_batch)

                # Loss
                loss_batches.append(
                    -np.mean(np.sum(labels_batch * np.log(activations[-1]), axis=1))
                )

                # Backward pass
                # Start with last layer
                delta = activations[-1] - labels_batch
                grad_w = np.dot(activations[-2].T, delta) / batch_size
                grad_b = np.mean(delta, axis=0)
                self.classification.W -= learning_rate * grad_w
                self.classification.b -= learning_rate * grad_b

                # Propagate error backwards through hidden layers
                for layer in range(1, self.num_layers):
                    if layer == 1:
                        delta = np.dot(delta, self.classification.W.T) * sigmoid_prime(
                            activations[-layer - 1]
                        )
                    else:
                        delta = np.dot(
                            delta, self.dbn.dbn[-layer + 1].W.T
                        ) * sigmoid_prime(activations[-layer - 1])
                    if layer == self.num_layers - 1:
                        grad_w = np.dot(data_batch.T, delta) / batch_size
                    else:
                        grad_w = np.dot(activations[-layer - 2].T, delta) / batch_size
                    grad_b = np.mean(delta, axis=0)
                    self.dbn.dbn[-layer].W -= learning_rate * grad_w
                    self.dbn.dbn[-layer].b -= learning_rate * grad_b

            # Compute cross-entropy loss
            previous_loss = train_loss
            train_loss = float(np.mean(loss_batches))
            loss.append(train_loss)

            if keep_track < early_stopping and round(train_loss, 3) == round(previous_loss, 3):
                keep_track += 1
            elif keep_track == early_stopping:
                return self
            # Print progress
            #if verbose:
             #   print(
               #     f"Epoch {epoch}/{epochs}: Train error -----------------{train_loss:.4f}"
              #  )
        if plot:
            plt.plot(np.arange(epochs), loss)
            plt.xlabel("Epochs")
            plt.ylabel("CrossEntropy Loss")
            if self.pretrained:
                plt.title("Loss for pretrained DNN")
            else:
                plt.title("Loss for DNN (without pretraining)")
            plt.show()

        self.fitted = True
        return self

    def test_dnn(self, test_data, test_labels, verbose=True):
        probs = self.entree_sortie_network(test_data)
        pred_label = np.argmax(probs[-1], axis=1)
        num_correct = np.sum(test_labels != pred_label)

        # Print the error rate and return it
        error_rate = num_correct / test_data.shape[0]

        if verbose:
            print(f"Error rate ----------------- : {error_rate:.2%}")
        return error_rate

    def plot_proba(self, data):
        pred_labels = self.entree_sortie_network(data)[-1]
        plt.scatter(np.arange(0, 10), pred_labels[0])
        plt.xlabel("Classes")
        plt.ylabel("Predicted probability for each class")
        plt.title("Probabilities by class")
        plt.show()

In [ ]:
def plot_loss(error_rate1, error_rate2, parameters, parameter_name):
    fig, axes = plt.subplots(1, 2, figsize=(15, 10))

    # Plot of the error rate by parameter for pretrained network
    axes[0].plot(parameters, error_rate1)
    axes[0].set_title("Impact of number of layers on pretrained network")
    axes[0].set_xlabel(parameter_name)
    axes[0].set_ylabel("Error rate")

    # Plot of the error rate by parameter for neural network
    axes[1].plot(parameters, error_rate2)
    axes[1].set_title("Impact of number of layers on neural network")
    axes[1].set_xlabel(parameter_name)
    axes[1].set_ylabel("Error rate")

    plt.suptitle(f"Comparing error rate of both models by {parameter_name}")
    plt.show()

In [ ]:
def import_model(filename: str) -> DNN:
    """
    :param filename: file path
    :return: trained model
    """
    with open("models/"+filename, "rb") as file:
        model = pickle.load(file)
        return model


import os
import pickle

def save_model(filename: str, model):
    """
    :param filename: file path
    :param model: trained model to save
    """
    directory = "models/"
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    with open(os.path.join(directory, filename), "wb") as file:
        pickle.dump(model, file)

# MNIST

In [ ]:
import numpy as np

def lire_mnist(filename: str, indices: np.ndarray, data_type: str):
    mnist_all = sp.io.loadmat(filename, simplify_cells=True)
    key = data_type + "0"
    data_mnist = (mnist_all[key] > 127).astype(int)
    label = np.zeros(mnist_all[key].shape[0])
    for i in indices[1:]:
        key = data_type + str(i)
        data_mnist = np.vstack([data_mnist, (mnist_all[key] > 127).astype(int)])
        y = i * np.ones(mnist_all[key].shape[0])
        label = np.concatenate([label, y], axis=0)
    return data_mnist, label

binaire = np.arange(0, 10)

file = "mnist_all.mat"

mnist_train, label_train = lire_mnist(file, binaire, "train")
mnist_test, label_test = lire_mnist(file, binaire, "test")
n_mnist, p_mnist = mnist_train.shape

# Paramètres

In [ ]:
epochs_rbm = 100
epochs_dnn = 100
learning_rate = 0.1
batch_size = 128
nb_gibbs = 200
digits = np.arange(0, 10)

# TEST 1 : DNN


• Spécifier les paramètres liés au réseau et à l’apprentissage : taille du réseau (vecteur contenant le nombre de neurones), nombre d’itérations pour les descentes de gradient (100 pour les RBM, 200 pour l’algorithme de rétro-propagation du gradient), learning rate (ex : 0.1), taille des mini-batch, le nombre de données d’apprentissage, ...

• Charger et binariser les données;

• Initialisation aléatoire du DNN;

• Si pré-apprentissage, pré-entraîner de manière non supervisée le DNN;

• EntraînerdemanièresuperviséleDNNpréalablementpré-entrainévial’algorithmederétro-propagation du gradient.

• Avec le réseau appris, observer les probabilités de sortie de quelques images de la base d’apprentissage.


## Train (with pretraining) DNN

In [ ]:
# Définissez les paramètres
config = (p_mnist, 200, 200)
train_model = True  # Change to False if you want to test instead of train

# Entraînement du modèle
if train_model:
    dnn = DNN(config)
    dnn.pretrain_dnn(mnist_train, epochs_rbm, learning_rate, batch_size)
    dnn.backward_propagation(mnist_train, label_train, epochs_dnn, learning_rate, batch_size,plot=False)

    # Sauvegarde du modèle
    path = f"dnn_testPretrained{dnn.pretrained}.pkl"
    save_model(path, dnn)

# Test du modèle
else:
    # Importez le modèle pré-entraîné
    path = "dnn_testPretrainedTrue.pkl"  # Assurez-vous que le nom de fichier correspond au modèle pré-entraîné
    dnn = import_model(path)

# Test du modèle
print(f"Modèle DNN avec pré-entraînement={dnn.pretrained}:")
dnn.test_dnn(mnist_test, label_test)

# Tracé des probabilités de la classe k
k = 2  # Choisissez une classe et tracez la probabilité
idx = np.where(label_test == k)[0]
dnn.plot_proba(mnist_test[idx])


# TEST 2 DNN : pretrained VS not pretrained 

deuxieme test : copier la partie correspondante TP 

1. initialiser deux réseaux identiques;
2. pré-apprendre un des deux réseau en le considérant comme un empilement de RBM (apprentissage non supervisé);
3. apprendre le réseau pré-appris préalablement avec l’algorithme de rétro-propagation;
4. apprendre le second réseau qui a été initialisé aléatoirement avec l’algorithme de rétro-propagation;
5. Calculer les taux de mauvaises classifications avec le réseau 1 (pré-entrainé + entraîné) et le réseau 2 (entraîné) à partir du jeu ’train’ et du jeu ’test’

Train 

In [ ]:
# Définissez les paramètres
config = (p_mnist, 200, 200)
train_models = True  # Change to False if you want to test instead of train

# Test 2 DNN := pretrained VS not pretrained (5.2)
if train_models:
    # 1
    dnn1 = DNN(config)  # to pretrain
    dnn2 = DNN(config)

    # 2
    dnn1.pretrain_dnn(mnist_train, epochs_rbm, learning_rate, batch_size)

    # 3
    dnn1.backward_propagation(mnist_train, label_train, epochs_dnn, learning_rate, batch_size, plot=False)

    # 4
    dnn2.backward_propagation(mnist_train, label_train, epochs_dnn, learning_rate, batch_size, plot=False)

    # Sauvegardez les deux modèles
    path_1 = f"dnn_testPretrained{dnn1.pretrained}.pkl"
    path_2 = f"dnn_testPretrained{dnn2.pretrained}.pkl"
    save_model(path_1, dnn1)
    save_model(path_2, dnn2)

else:
    # Importez les modèles
    path_1 = "dnn_testPretrainedTrue.pkl"
    dnn1 = import_model(path_1)
    path_2 = "dnn_testPretrainedFalse.pkl"
    dnn2 = import_model(path_2)

# 5
print(f"Modèle DNN avec pré-entraînement={dnn1.pretrained} pour le jeu de données d'entraînement:")
dnn1.test_dnn(mnist_train, label_train)

print(f"Modèle DNN avec pré-entraînement={dnn1.pretrained} pour le jeu de données de test:")
dnn1.test_dnn(mnist_test, label_test)

print(f"Modèle DNN avec pré-entraînement={dnn2.pretrained} pour le jeu de données d'entraînement:")
dnn2.test_dnn(mnist_train, label_train)

print(f"Modèle DNN avec pré-entraînement={dnn2.pretrained} pour le jeu de données de test:")
dnn2.test_dnn(mnist_test, label_test)



# ANALYSE : GRAPH

Test with different number layers

2 courbes exprimant le taux d’erreur des 2 réseaux en fonction du nombre de couches (par exemple 2 couches de 200, puis 3 couches de 200, ... puis 5 couches de 200);

In [ ]:
# Définissez les paramètres
config = (p_mnist, 200, 200)
train_models = True  # Change to False if you want to test instead of train
test_nb_layers = True  # Change to False if you don't want to test with different number of layers

# Test avec différents nombres de couches (5.2.2)
if test_nb_layers:
    error_rate1, error_rate2 = [], []
    nb_layers = [2, 3, 4, 5]

    for layer in nb_layers:
        # Définissez la configuration pour chaque itération
        config = [p_mnist]
        config.extend([200 for _ in range(layer)])

        if train_models:
            dnn1 = DNN(config)  # pour pré-entraîner
            dnn2 = DNN(config)

            # Pré-entraîner le modèle puis l'entraîner
            dnn1.pretrain_dnn(mnist_train, epochs_rbm, learning_rate, batch_size)
            dnn1.backward_propagation(mnist_train, label_train, epochs_dnn, learning_rate, batch_size, plot=False)

            # Entraîner le modèle directement
            dnn2.backward_propagation(mnist_train, label_train, epochs_dnn, learning_rate, batch_size, plot=False)

            # Sauvegardez les deux modèles
            path_1 = f"dnn_testPretrained{dnn1.pretrained}_NbLayers{str(layer)}.pkl"
            path_2 = f"dnn_testPretrained{dnn2.pretrained}_NbLayers{str(layer)}.pkl"
            save_model(path_1, dnn1)
            save_model(path_2, dnn2)

        else:
            # Importez les modèles
            path_1 = f"dnn_testPretrainedTrue_NbLayers{str(layer)}.pkl"
            path_2 = f"dnn_testPretrainedFalse_NbLayers{str(layer)}.pkl"
            dnn1 = import_model(path_1)
            dnn2 = import_model(path_2)

        # Calculez le taux d'erreur pour les deux modèles
        print(f"Modèle DNN avec pré-entraînement={dnn1.pretrained} pour nb_layer={layer}:")
        error1 = dnn1.test_dnn(mnist_test, label_test)

        print(f"Modèle DNN avec pré-entraînement={dnn2.pretrained} pour nb_layer={layer}:")
        error2 = dnn2.test_dnn(mnist_test, label_test)

        error_rate1.append(error1)
        error_rate2.append(error2)

    # Tracer le taux d'erreur en fonction du nombre de couches
    plot_loss(error_rate1, error_rate2, nb_layers, "Nombre de couches")


Test with different number neurons

2 courbes exprimant le taux d’erreur des 2 réseaux en fonction du nombre de neurones par couches (par exemple 2 couches de 100, puis 2 couches de 300, ...puis 2 couches de 700,...);

In [ ]:
# Définissez les paramètres
config = (p_mnist, 200, 200)
train_models = True  # Change to False if you want to test instead of train
test_nb_neurons = True  # Change to False if you don't want to test with different number of neurons

# Test avec différents nombres de neurones (5.2.2)
if test_nb_neurons:
    error_rate1, error_rate2 = [], []
    nb_neurons = [100, 200, 300, 700]

    for neurons in nb_neurons:
        # Définissez la configuration pour chaque itération
        config = [p_mnist]
        config.extend([neurons for _ in range(2)])

        if train_models:
            dnn1 = DNN(config)  # pour pré-entraîner
            dnn2 = DNN(config)

            # Pré-entraîner le modèle puis l'entraîner
            dnn1.pretrain_dnn(mnist_train, epochs_rbm, learning_rate, batch_size)
            dnn1.backward_propagation(mnist_train, label_train, epochs_dnn, learning_rate, batch_size, plot=False)

            # Entraîner le modèle directement
            dnn2.backward_propagation(mnist_train, label_train, epochs_dnn, learning_rate, batch_size, plot=False)

            # Sauvegardez les deux modèles
            path_1 = f"dnn_testPretrained{dnn1.pretrained}_NbNeurons{str(neurons)}.pkl"
            path_2 = f"dnn_testPretrained{dnn2.pretrained}_NbNeurons{str(neurons)}.pkl"
            save_model(path_1, dnn1)
            save_model(path_2, dnn2)

        else:
            # Importez les modèles
            path_1 = f"dnn_testPretrainedTrue_NbNeurons{str(neurons)}.pkl"
            path_2 = f"dnn_testPretrainedFalse_NbNeurons{str(neurons)}.pkl"
            dnn1 = import_model(path_1)
            dnn2 = import_model(path_2)

        # Calculez le taux d'erreur pour les deux modèles
        print(f"Modèle DNN avec pré-entraînement={dnn1.pretrained} pour nb_neurons={neurons}:")
        error1 = dnn1.test_dnn(mnist_test, label_test)

        print(f"Modèle DNN avec pré-entraînement={dnn2.pretrained} pour nb_neurons={neurons}:")
        error2 = dnn2.test_dnn(mnist_test, label_test)

        error_rate1.append(error1)
        error_rate2.append(error2)

    # Tracer le taux d'erreur en fonction du nombre de neurones
    plot_loss(error_rate1, error_rate2, nb_neurons, "Nombre de neurones")


Test with different size of training data

2 courbes exprimant le taux d’erreur des 2 réseaux en fonction du nombre de données train (par exemple on fixe 2 couches de 200 puis on utilise 1000 données train, 3000, 7000, 10000, 30000, 60000).

In [ ]:
# Définissez les paramètres
config = (p_mnist, 200, 200)
train_models = True  # Change to False if you want to test instead of train
test_train_size = True  # Change to False if you don't want to test with different sizes of training data

# Test avec différentes tailles de données d'entraînement (5.2.2)
if test_train_size:
    error_rate1, error_rate2 = [], []
    train_sizes = [1000, 3000, 7000, 10000, 30000, 60000]

    for size in train_sizes:
        config = (p_mnist, 200, 200)

        if train_models:
            dnn1 = DNN(config)  # pour pré-entraîner
            dnn2 = DNN(config)

            # Sélectionnez les données d'entraînement pour chaque itération
            data_shuffled, labels_shuffled = shuffle(mnist_train, label_train)
            data_sampled, labels_sampled = data_shuffled[:size], labels_shuffled[:size]

            # Pré-entraîner le modèle puis l'entraîner
            dnn1.pretrain_dnn(data_sampled, epochs_rbm, learning_rate, batch_size)
            dnn1.backward_propagation(data_sampled, labels_sampled, epochs_dnn, learning_rate, batch_size, plot=False)

            # Entraîner le modèle directement
            dnn2.backward_propagation(data_sampled, labels_sampled, epochs_dnn, learning_rate, batch_size, plot=False)

            # Sauvegardez les deux modèles
            path_1 = f"dnn_testPretrained{dnn1.pretrained}_TrainSize{str(size)}.pkl"
            path_2 = f"dnn_testPretrained{dnn2.pretrained}_TrainSize{str(size)}.pkl"
            save_model(path_1, dnn1)
            save_model(path_2, dnn2)

        else:
            # Importez les modèles
            path_1 = f"dnn_testPretrainedTrue_TrainSize{str(size)}.pkl"
            path_2 = f"dnn_testPretrainedFalse_TrainSize{str(size)}.pkl"
            dnn1 = import_model(path_1)
            dnn2 = import_model(path_2)

        # Calculez le taux d'erreur pour les deux modèles
        print(f"Modèle DNN avec pré-entraînement={dnn1.pretrained} pour train_size={size}:")
        error1 = dnn1.test_dnn(mnist_test, label_test)

        print(f"Modèle DNN avec pré-entraînement={dnn2.pretrained} pour train_size={size}:")
        error2 = dnn2.test_dnn(mnist_test, label_test)

        error_rate1.append(error1)
        error_rate2.append(error2)

    # Tracer le taux d'erreur en fonction de la taille de l'échantillon
    plot_loss(error_rate1, error_rate2, train_sizes, "Taille de l'échantillon")
